<a href="https://colab.research.google.com/github/RZx-Taishou/Facial-Pain-Recognition/blob/main/Facial_Pain_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB7

In [ ]:
!git clone https://github.com/RZx-Taishou/Facial-Pain-Recognition.git

fatal: destination path 'Facial-Pain-Recognition' already exists and is not an empty directory.


In [ ]:
ORIGINAL_DIR = "/content/Facial-Pain-Recognition/Dataset"

In [ ]:
training_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=.2
)

validation_datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=.2
)

train_generator = training_datagen.flow_from_directory(
    ORIGINAL_DIR,
    target_size=(150, 150),
    class_mode="binary",
    batch_size=64,
    subset="training"
)

validation_generator = validation_datagen.flow_from_directory(
    ORIGINAL_DIR,
    target_size=(150, 150),
    class_mode="binary",
    batch_size=32,
    subset="validation"
)

Found 718 images belonging to 2 classes.
Found 178 images belonging to 2 classes.


In [ ]:
effb7 = EfficientNetB7(input_shape=(150, 150, 3), include_top=False)

for layer in effb7.layers:
  layer.trainable = False

x = effb7.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(.2)(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = Model(effb7.input, x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_6 (Rescaling)        (None, 150, 150, 3)  0           ['input_4[0][0]']                
                                                                                                  
 normalization_3 (Normalization  (None, 150, 150, 3)  7          ['rescaling_6[0][0]']            
 )                                                                                                
                                                                                            

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_generator, epochs=100, validation_data=validation_generator, verbose=2)

In [ ]:
model.save("FPR.h5")

In [ ]:
from google.colab import files
files.download("FPR.h5")